In [391]:
# 7.1)

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(0)

train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")

print('As colunas do banco de dados de treino são:')
print(list(train))
print('\nAs colunas do banco de dados de teste são:')
print(list(test))

In [392]:
print('Tipos de dado no treino:')
train.info()
print('\nTipos de dado no teste:')
test.info()

In [393]:
print('Dados faltantes no treino:')
print(train.isnull().sum())
treino = train.copy()

print('\nDados faltantes no teste:')
print(test.isnull().sum())
teste = test.copy()

# 7.2)

treino = treino.drop(['Cabin','Ticket','Name','PassengerId'],axis=1)
medianaIdade = treino['Age'].median()
medianaTarifa = treino['Fare'].median()
modaEmbarco = treino['Embarked'].mode()[0]
treino['Age'] = treino['Age'].fillna(medianaIdade)
treino['Fare'] = treino['Fare'].fillna(medianaTarifa)
treino['Embarked'] = treino['Embarked'].fillna(modaEmbarco)

teste = teste.drop(['Cabin','Ticket','Name','PassengerId'],axis=1)
teste['Age'] = teste['Age'].fillna(medianaIdade)
teste['Fare'] = teste['Fare'].fillna(medianaTarifa)
teste['Embarked'] = teste['Embarked'].fillna(modaEmbarco)

In [394]:
# 7.3)

treino['AgeSpaced'],retbinsAge = pd.cut(treino['Age'],5,retbins=True)
retbinsAge[0] = 0
treino['FareBalanced'],retbinsFare = pd.qcut(treino['Fare'],6,retbins=True)

teste['AgeSpaced'] = pd.cut(teste['Age'],retbinsAge)
teste['FareBalanced'] = pd.cut(teste['Fare'],retbinsFare,include_lowest=True)
teste.info()

In [395]:
# 8.1)
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

def onehot_encode(treino,teste,colunas):
    treinoAper = treino.copy()
    testeAper = teste.copy()
    
    enc = OneHotEncoder(handle_unknown = 'ignore',sparse = False)
    treinoCoded = pd.DataFrame(enc.fit_transform(treinoAper[colunas]))
    testeCoded = pd.DataFrame(enc.transform(testeAper[colunas]))
    
    treinoCoded.columns = enc.get_feature_names_out(colunas)
    testeCoded.columns = enc.get_feature_names_out(colunas)
    
    treinoAper = treinoAper.select_dtypes(exclude=['object'])
    testeAper = testeAper.select_dtypes(exclude=['object'])
    
    treinoAper = pd.concat([treinoAper, treinoCoded], axis=1)
    testeAper = pd.concat([testeAper, testeCoded], axis=1)
    return treinoAper, testeAper

def label_encode(treino,teste,colunas):
    treinoAper = treino.copy()
    testeAper = teste.copy()
    
    enc = LabelEncoder()
    treinoAper[colunas] = treino[colunas].apply(enc.fit_transform)
    testeAper[colunas] = teste[colunas].apply(enc.fit_transform)
    return treinoAper, testeAper

print('Tipos de dado no treino:')
treino.info()
print('\nTipos de dado no teste:')
teste.info()

treinoAper = treino.drop(['Age','Fare'],axis=1)
testeAper = teste.drop(['Age','Fare'],axis=1)

colunas = list(testeAper.select_dtypes(include=['object']))
treinoAper, testeAper = onehot_encode(treinoAper,testeAper,colunas)

colunas = list(testeAper.select_dtypes(include=['category']))
treinoAper, testeAper = label_encode(treinoAper,testeAper,colunas)

In [396]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = treinoAper.drop(['Survived'],axis=1)
y = treinoAper['Survived']

Xtrain,Xval,ytrain,yval = train_test_split(X,y,test_size=0.20,random_state=0)

Xtest = testeAper

modelo = RandomForestClassifier(n_estimators=150,max_depth=8,random_state=0)
modelo.fit(Xtrain,ytrain)
prev = modelo.predict(Xval)
AC = accuracy_score(prev,yval)
print('A acurácia foi de: %.2f%%' %(AC*100))

In [397]:
# 8.2)
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

def CeApipelines(train,test,encoder,model,
                 numerical_inputer,categorical_imputer):
    treinoPipe = train.drop(['Cabin','Ticket','Name','PassengerId'],axis=1)
    testePipe = test.drop(['Cabin','Ticket','Name','PassengerId'],axis=1)
    X = treinoPipe.drop(['Survived'],axis=1)
    y = treinoPipe['Survived']

    Xtrain,Xval,ytrain,yval = train_test_split(X,y,test_size=0.20,random_state=0)
    Xtest = testePipe
    
    cat = list(Xtest.select_dtypes(include=['object']))
    num = list(Xtest.select_dtypes(include=['number']))
    
    catT = Pipeline(steps = [('inp', SimpleImputer(strategy=categorical_imputer)),
                             ('enc', encoder)])
    t = [('num',SimpleImputer(strategy=numerical_inputer), num),
         ('cat',catT, cat)]
    colt = ColumnTransformer(transformers=t)
    
    pipe = Pipeline([('colt', colt),('model', model)])
    
    pipe.fit(Xtrain, ytrain)
    prev = pipe.predict(Xval)
    AC = accuracy_score(prev, yval)
    return AC*100

acc = CeApipelines(train,test,
                   OneHotEncoder(handle_unknown = 'ignore',sparse = False),
                   RandomForestClassifier(n_estimators=150,
                                          max_depth=8,random_state=0),
                   'mean','most_frequent')
print('A acurácia básica foi de: %.2f%%' %acc)

encoders = [OneHotEncoder(handle_unknown = 'ignore',sparse = False),
            OrdinalEncoder()]
num_inputs = ['mean','most_frequent','median','constant']

for enc in encoders:
    print('\nEncoder:')
    print(enc)
    for num in num_inputs:
        acc = CeApipelines(train,test,
                           enc,
                           RandomForestClassifier(n_estimators=150,
                                                  max_depth=8,random_state=0),
                           num,'most_frequent')
        print('A acurácia para input %s foi de: %.2f%%' %(num,acc))

print('\nA maior acurácia foi obtida com a mediana e o one-hot encoder')
print('estando igual a média com ordinal encoder, entretanto, todas')
print('estão com valores bem próximos.')

In [400]:
# 8.3)

from sklearn.ensemble import GradientBoostingClassifier

acc = CeApipelines(train,test,
                   OrdinalEncoder(),
                   GradientBoostingClassifier(n_iter_no_change = 100,random_state=0),
                   'mean','most_frequent')
print('A acurácia do XGBoost foi de: %.2f%%' %acc)

encoders = [OneHotEncoder(handle_unknown = 'ignore',sparse = False),
            OrdinalEncoder()]
num_inputs = ['mean','most_frequent','median','constant']

for enc in encoders:
    print('\nEncoder:')
    print(enc)
    for num in num_inputs:
        acc = CeApipelines(train,test,
                           enc,
                           GradientBoostingClassifier(n_iter_no_change = 100,random_state=0),
                           num,'most_frequent')
        print('A acurácia para input %s foi de: %.2f%%' %(num,acc))
        
print('\nEmbora a acurácia não tenha sido mais alta para os mesmos parâmetros da random forest,')
print('ao testar a mesma variação de parâmetros houve uma acurácia maior de 86%,')
print('mas eu havia esquecido de definir o random_state=0 no modelo.')
print('Após essa mudança o modelo performou pior que o random forest.')